1. Написати функцію пошуку випадкового простого числа з заданого інтервалу або
заданої довжини, використовуючи датчик випадкових чисел та тести перевірки на
простоту. В якості датчика випадкових чисел використовуйте вбудований генератор
псевдовипадкових чисел вашої мови програмування. В якості тесту перевірки на простоту
рекомендовано використовувати тест Міллера-Рабіна із попередніми пробними
діленнями. Тести необхідно реалізовувати власноруч, використання готових реалізацій
тестів не дозволяється.

In [48]:
import random

#Перевірка числа на простоту за допомогою тесту Міллера-Рабіна
def miller_rabin(n, k=10):
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0:
        return False

    # Представимо n-1 у вигляді (2^r) * d
    r, d = 0, n - 1
    while d % 2 == 0:
        r += 1
        d //= 2

    # k ітерацій тесту 
    for _ in range(k):
        a = random.randint(2, n - 2)
        x = pow(a, d, n)  # a^d % n
        if x == 1 or x == n - 1:
            continue
        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True

 # Генеруємо випадкове просте число
def generate_number(bit_length):
    while True:
        number = random.getrandbits(bit_length)
        number |= (1 << bit_length - 1) | 1 
        if miller_rabin(number):
            return number

bit_length = 256
prime_number = generate_number(bit_length)
print(f"Згенероване просте число ({bit_length} біт): {prime_number}")


Згенероване просте число (256 біт): 81575641111645250922011054406759458687687729611391087788670614759804575094387


2. За допомогою цієї функції згенерувати дві пари простих чисел p, q  і p1, q1 довжини щонайменше 256 біт. При цьому пари чисел беруться так, щоб pq < p1q1 ; p і q – прості числа для побудови ключів абонента А, p1 і q1 – абонента B. 

In [49]:
p = generate_number(bit_length)
q = generate_number(bit_length)
p1 = generate_number(bit_length)
q1 = generate_number(bit_length)

while p * q <= p1 * q1:
    p1 = generate_number(bit_length)
    q1 = generate_number(bit_length)

print(f"Абонент А:  p = {p}, q = {q}")

print(f"Абонент В:  p1 = {p1}, q1 = {q1}")


Абонент А:  p = 62134537171254466919512511820159991692932331059326494868784649327292291000337, q = 78784354574222019020508201870722725030755245451139870572001567965135149491103
Абонент В:  p1 = 59008500315852422418732402544458578100965240199090740535226129290215080363447, q1 = 67434160788590422198386831741757576284306511871834137029420064400676972866377


3. Написати функцію генерації ключових пар для RSA. Після генерування функція повинна повертати та/або зберігати секретний ключ (d, p,q) та відкритий ключ (n,e) . За допомогою цієї функції побудувати схеми RSA для абонентів А і B – тобто, створити та зберегти для подальшого використання відкриті ключі (e,n) , (e1, n1) та секретні d і d1 . 

In [50]:
# a^(-1) mod m
def euclidean_algorithm(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        gcd, x, y = euclidean_algorithm(b % a, a)
        return (gcd, y - (b // a) * x, x)

def mod_inverse(a, m):
    gcd, x, y = euclidean_algorithm(a, m)
    if gcd != 1:
        raise ValueError('Обернене за модулем не існує')
    else:
        return x % m


In [51]:
#Генеруємо ключі RSA
def generate_key(p, q):    
    n = p * q
    phi = (p - 1) * (q - 1)
    e = 65537  
    d = mod_inverse(e, phi)  
    
    return (n, e), (d, p, q)

public_key_A, private_key_A = generate_key(p, q)
public_key_B, private_key_B = generate_key(p1, q1)

print(f"Відкритий ключ A: n = {public_key_A[0]}, e = {public_key_A[1]}")
print(f"Секретний ключ A: d = {private_key_A[0]}, p = {private_key_A[1]}, q = {private_key_A[2]}")

print(f"\nВідкритий ключ B: n = {public_key_B[0]}, e = {public_key_B[1]}")
print(f"Секретний ключ B: d = {private_key_B[0]}, p = {private_key_B[1]}, q = {private_key_B[2]}")



Відкритий ключ A: n = 4895229407805289931250719627411865709319332861859916814970749665052652135224278294245020857551998749713106923958720977276518042356504207821835532351501711, e = 65537
Секретний ключ A: d = 1420383637011541470202537260400446134678371510766867237644136527925312922523493543032453188982592623871292482723619031401122176252285915211742857973171969, p = 62134537171254466919512511820159991692932331059326494868784649327292291000337, q = 78784354574222019020508201870722725030755245451139870572001567965135149491103

Відкритий ключ B: n = 3979188698192780967144092338772336218425554528630557210993023110870913544846498123495094871221791469861666759731379593518567160268249845439274122526121519, e = 65537
Секретний ключ B: d = 591441126525411285242696545035343807371154106281798950093581307093604663630445497093304557163465757502804734348284201112767441734297641421658395227679601, p = 59008500315852422418732402544458578100965240199090740535226129290215080363447, q = 6743416078859042

4. Написати програму шифрування, розшифрування і створення повідомлення з цифровим підписом для абонентів А і B. Кожна з операцій (шифрування, розшифрування, створення цифрового підпису, перевірка цифрового підпису) повинна бути реалізована окремою процедурою, на вхід до якої повинні подаватись лише ті ключові дані, які необхідні для її виконання. За допомогою датчика випадкових чисел вибрати відкрите повідомлення M і знайти криптограму для абонентів А и B, перевірити правильність розшифрування. Скласти для А і B повідомлення з цифровим підписом і перевірити його. 

In [52]:

# Функція піднесення до степеня за схемою Горнера
def mod_exp(x, alpha, m):
    y = 1
    while alpha > 0:
        if alpha % 2 == 1:  
            y = (y * x) % m
        x = (x * x) % m  
        alpha //= 2 
    return y

# Зашифрування 
def encrypt_RSA(M, e, n):
    return mod_exp(M, e, n)

# Розшифрування 
def decrypt_RSA(C, d, n):
    return mod_exp(C, d, n)

# Створення цифрового підпису 
def sign(M, d, n):
    return mod_exp(M, d, n)

# Перевірка цифрового підпису 
def verify_signature(M, S, e, n):
    return M == mod_exp(S, e, n)

# Відкритте повідомлення M
M = random.randint(0, public_key_A[0] - 1)

# Зашифрування та розшифрування для абонента A
C_A = encrypt_RSA(M, public_key_A[1], public_key_A[0])  # зашифроване повідомлення для A
M_decrypted_A = decrypt_RSA(C_A, private_key_A[0], public_key_A[0])  # розшифроване повідомлення для A
# Перевірка правильності розшифрування для абонента A
print(f"Абонент A:")
print(f"Повідомлення М: {M}")
print(f"Зашифроване повідомлення С: {C_A}")
print(f"Розшифроване повідомлення М: {M_decrypted_A}")
print(f"Правильність розшифрування для A: {M == M_decrypted_A}")
# Створення та перевірка цифрового підпису для абонента A
S_A = sign(M, private_key_A[0], public_key_A[0])  # цифровий підпис для A
is_valid_A = verify_signature(M, S_A, public_key_A[1], public_key_A[0])  # перевірка підпису для A
print(f"Цифровий підпис S: {S_A}")
print(f"Перевірка підпису M: {is_valid_A}\n")

# Зашифрування та розшифрування для абонента B
C_B = encrypt_RSA(M, public_key_B[1], public_key_B[0])  # зашифроване повідомлення для B
M_decrypted_B = decrypt_RSA(C_B, private_key_B[0], public_key_B[0])  # розшифроване повідомлення для B
# Перевірка правильності розшифрування для абонента B
print(f"Абонент B:")
print(f"Повідомлення М: {M}")
print(f"Зашифроване повідомлення С: {C_B}")
print(f"Розшифроване повідомлення М: {M_decrypted_B}")
print(f"Правильність розшифрування для B: {M == M_decrypted_B}")
# Створення та перевірка цифрового підпису для абонента B
S_B = sign(M, private_key_B[0], public_key_B[0])  # цифровий підпис для B
is_valid_B = verify_signature(M, S_B, public_key_B[1], public_key_B[0])  # перевірка підпису для B
print(f"Цифровий підпис S: {S_B}")
print(f"Перевірка підпису M: {is_valid_B}")


Абонент A:
Повідомлення М: 929079210764478323906376297741595423638273323217476632326397738993690992839182309411913314276024984055527686886729281750654008904704003795323600975772280
Зашифроване повідомлення С: 3791645558981838178502684831739811286738678042039983860531097152247123220264306802628585834837819143843417660157896979960710641514558703083191973424031169
Розшифроване повідомлення М: 929079210764478323906376297741595423638273323217476632326397738993690992839182309411913314276024984055527686886729281750654008904704003795323600975772280
Правильність розшифрування для A: True
Цифровий підпис S: 3469901439187506820480308345659763932419786709951998386517249785327860061275848058447289420092864341428846925395623433737789401931885383012374032354125127
Перевірка підпису M: True

Абонент B:
Повідомлення М: 929079210764478323906376297741595423638273323217476632326397738993690992839182309411913314276024984055527686886729281750654008904704003795323600975772280
Зашифроване повідомлення С: 2150

5. За допомогою раніше написаних на попередніх етапах програм організувати роботу протоколу конфіденційного розсилання ключів з підтвердженням справжності по відкритому каналу за допомогою алгоритму RSA. Протоколи роботи кожного учасника (відправника та приймаючого) повинні бути реалізовані у вигляді окремих процедур, на вхід до яких повинні подаватись лише ті ключові дані, які необхідні для виконання. Перевірити роботу програм для випадково обраного ключа 0 < k < n. 

In [53]:
def generate_session_key(bit_length):
    return random.getrandbits(bit_length)

# Відправник: створення повідомлення для передачі
def sender_protocol(session_key, recipient_public_key, sender_private_key, sender_public_key):
    encrypted_session_key = encrypt_RSA(session_key, recipient_public_key[1], recipient_public_key[0])
    signature = sign(session_key, sender_private_key[0], sender_public_key[0])
    return encrypted_session_key, signature

# Отримувач: знаходить (конфіденційність):
def recipient_protocol(encrypted_session_key, signature, sender_public_key, recipient_private_key, recipient_public_key):
    session_key = decrypt_RSA(encrypted_session_key, recipient_private_key[0], recipient_public_key[0])
    is_valid = verify_signature(session_key, signature, sender_public_key[1], sender_public_key[0])
    S = mod_exp(session_key, recipient_private_key[0], recipient_public_key[0])
    return session_key, is_valid, S

session_key_bit_length = 256
session_key = generate_session_key(session_key_bit_length)
print(f"Ключ k1: {session_key}")

encrypted_key_A, signature_A = sender_protocol(session_key, public_key_B, private_key_A, public_key_A)
print("\nВідправник (A) відправляє зашифрований ключ k1 та підпис s1.")
print(f"Зашифрований ключ k1: {encrypted_key_A}")
print(f"Цифровий підпис s1: {signature_A}")

received_session_key, is_valid_signature, S = recipient_protocol(encrypted_key_A, signature_A, public_key_A, private_key_B, public_key_B)

print("\nОтримувач (B) прийняв повідомлення.")
print(f"Розшифрований ключ k: {received_session_key}")
print(f"Підпис дійсний: {is_valid_signature}")
print(f"Обчислений S: {S}")
print(f"Ключ k1 переданий успішно: {received_session_key == session_key}")

Ключ k1: 10916774011550799312414644699893603725611046078698185845218623998006298755787

Відправник (A) відправляє зашифрований ключ k1 та підпис s1.
Зашифрований ключ k1: 2977570074230823852640051443870707573271611928598368481750147533014591519200566575797307546990782536830679234901318527344919456681466172240881721987056528
Цифровий підпис s1: 2567057846588635624032509946794259440743229679471759187200624878641567310880226264760762055476298879402647192662259455419210092975177138279001929108908896

Отримувач (B) прийняв повідомлення.
Розшифрований ключ k: 10916774011550799312414644699893603725611046078698185845218623998006298755787
Підпис дійсний: True
Обчислений S: 3591106944979327643384837905779963146966863073010426266237661933506154946116249953003793104148951931179114633603893579639090387449636651989458010017955163
Ключ k1 переданий успішно: True
